# Tasks 

## Task 1: Binary Representatios

Focusing on implemnet the four key bitwise operations, 


1. `rotl(x, n=1)` – Bit rotation to the left.
2. `rotr(x, n=1)` – Bit rotation to the right.
3. `ch(x, y, z)` – Choose function: bits from y or z depending on x.
4. `maj(x, y, z)` – Majority function based on bit voting.

All operations assume 32-bit unsigned integrers 

### Creating the function for the bit rotation to the right

In [20]:
# Function for roating left 
def rotl(x, n=1, bits=32):
    return ((x << n) & (2**bits - 1)) | (x >> (bits - n))

### rotl(x, n)

The `rotl` function performs a circular left shift on a 32-bit integer.
- Bits that go past the left edge wrap around to the right side.
- We apply a bitmask `& (2**32 - 1)` to ensure the result stays within 32 bits.

Example:
- `rotl(0b1001, 1)` → shifts left and wraps `1` around.


### Creating the function for bit rotation to the left

In [21]:
# Function for roating right
def rotr(x, n=1, bits=32):
    return (x >> n) | ((x << (bits - n)) & (2**bits - 1))

### rotr(x, n)

`rotr` rotates the bits to the right — the inverse of `rotl`.
- High bits from the left wrap around to the right.
- Useful in SHA and cryptographic transforms.

Example:
- `rotr(0b1001, 1)` → last bit comes to front.


In [22]:
def ch(x, y, z):
    return (x & y) ^ (~x & z)



### ch(x, y, z)

This function is often used in SHA-2.
- It's a conditional selector at the bit level.
- If a bit in `x` is 1 → result takes the corresponding bit from `y`.
- If 0 → it takes it from `z`.

It's like `x ? y : z` for each bit.


In [23]:
def maj(x, y, z):
    return (x & y) ^ (x & z) ^ (y & z)


### maj(x, y, z)

This function returns the majority bit for each position:
- If 2 or more of `x`, `y`, `z` have a `1`, the result has a `1` in that bit.
- It's used in SHA256 to mix entropy.

Mathematically: `maj(x, y, z) = (x & y) ^ (x & z) ^ (y & z)`


### Test for part 1 

In [24]:
# Example test values
x = 0b10101010_11110000_00001111_11000011
y = 0b11001100_00110011_11110000_00001111
z = 0b11110000_00001111_10101010_01010101

print(f"rotl(x, 3): {rotl(x, 3):032b}")
print(f"rotr(x, 3): {rotr(x, 3):032b}")
print(f"ch(x, y, z): {ch(x, y, z):032b}")
print(f"maj(x, y, z): {maj(x, y, z):032b}")


rotl(x, 3): 01010111100000000111111000011101
rotr(x, 3): 01110101010111100000000111111000
ch(x, y, z): 11011000001111111010000000010111
maj(x, y, z): 11101000001100111010101001000111


## Task 2: Hash Functions 

The focus fo this task is to ranslate classic C hash function into python. The function uses a prime multiplier and a modulus to generate a small hash value for strings. We will test the output of the converted function and explain why the constants `31` and `101` are used.


### The Original C Function

The following function appears in *The C Programming Language* by Kernighan and Ritchie:

```c
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
} 
```


In [25]:

def simple_hash(s):
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval
    return hashval % 101


### How the Hash Function Works

- For each character in the string:
  - Convert the character to its ASCII code using `ord(char)`
  - Multiply the current hash value by 31 and add the character's ASCII code
- Finally, take the result modulo 101 to ensure the hash is within a small range

This method is used in hash tables for quick indexing.


In [26]:
# Sample test cases
test_inputs = ["abc", "hello", "world", "hash", "function", "test", "python", ""]

# Display hash values for each string
for string in test_inputs:
    print(f"Hash('{string}') = {simple_hash(string)}")


Hash('abc') = 0
Hash('hello') = 17
Hash('world') = 34
Hash('hash') = 15
Hash('function') = 100
Hash('test') = 86
Hash('python') = 91
Hash('') = 0


### Why Use 31 and 101?

- **31**:
  - A small prime number, often used in hash functions.
  - Prime multipliers reduce the likelihood of collisions.
  - 31 is also efficient computationally: `2^5 - 1`, so compilers may optimize it.

- **101**:
  - Another small prime, used to constrain the hash output.
  - In real applications, this would be the size of a hash table (e.g., 101 buckets).
  - Using a prime as the modulus helps ensure a better distribution of hash values.


## Task 3: SHA256 Padding

This task involves implementing the padding step of the SHA256 hashing process, based on the FIPS 180-4 standard.

Padding consists of three steps:
1. Append a single `1` bit (in hex: `0x80`)
2. Append `0` bits until the total length (in bits) is congruent to 448 modulo 512
3. Append the original message length (in bits) as a 64-bit big-endian integer

We'll implement this padding logic in Python, apply it to a file, and print the final padding in hexadecimal format.


In [27]:
import os

def sha256_padding(filepath):
    with open(filepath, 'rb') as f:
        message = f.read()
    
    original_length_bits = len(message) * 8
    padded = bytearray(message)
    
    # Step 1: Append the '1' bit (in hex: 0x80)
    padded.append(0x80)
    
    # Step 2: Append zeros until length ≡ 448 mod 512 (i.e., length ≡ 56 mod 64 bytes)
    while (len(padded) % 64) != 56:
        padded.append(0x00)
    
    # Step 3: Append 64-bit big-endian representation of original length
    padded += original_length_bits.to_bytes(8, byteorder='big')
    
    # Get only the padding part
    padding_only = padded[len(message):]
    
    # Print the padding in hex
    print("Padding (hex):")
    print(" ".join(f"{byte:02x}" for byte in padding_only))


### SHA256 Padding Explanation

- A single `1` bit (hex `0x80`) is added to mark the end of the message.
- The padding with zeros ensures the total message length is a multiple of 512 bits (64 bytes).
  - Specifically, the message length should be congruent to `448 mod 512`, so the final 64 bits can store the original message length.
- The original message length in **bits** is added at the end as an **8-byte big-endian integer**.


In [28]:
# Write "abc" to a temporary test file
test_file = "abc_test.txt"
with open(test_file, "wb") as f:
    f.write(b"abc")

# Run the padding function
sha256_padding(test_file)


Padding (hex):
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


### Expected Output (for "abc")

According to the FIPS 180-4 specification, the input "abc" (3 bytes = 24 bits) should yield this padding:

80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


This shows:
- `0x80` as the `1` bit
- 56 total bytes before the final 8-byte length
- `0x18` = 24 bits, stored as a big-endian integer


## Task 4: Prime Numbers

In this task, we calculate the first 100 prime numbers using two different algorithms:

1. **Trial Division Algorithm**
2. **Sieve of Eratosthenes**

We explain how each algorithm works

---


### Method 1: Trial Division

The Trial Division algorithm checks each number individually to determine if it is prime.
For each candidate number:
- Check divisibility by all integers from 2 up to the square root of the candidate.
- If no divisors are found, the number is prime.

This method is simple but inefficient for large numbers.

> **Reference:** [Trial Division Algorithm - GeeksforGeeks](https://www.geeksforgeeks.org/prime-numbers/)

---


In [8]:
import math

def is_prime_trial(n):
    """Check if a number is prime using Trial Division."""
    if n < 2:
        return False
    for i in range(2, int(math.isqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def generate_primes_trial(limit):
    """Generate the first `limit` prime numbers using Trial Division."""
    primes = []
    candidate = 2
    while len(primes) < limit:
        if is_prime_trial(candidate):
            primes.append(candidate)
        candidate += 1
    return primes

# First 100 primes using Trial Division
primes_trial = generate_primes_trial(100)
print(primes_trial)


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


### Method 2: Sieve of Eratosthenes

The Sieve of Eratosthenes is an efficient algorithm for finding all primes up to a given limit.
Steps:
- Create a boolean list marking all numbers as potential primes.
- Starting from 2, eliminate all multiples of each prime found.
- Remaining unmarked numbers are primes.

This method is much faster for finding multiple primes at once.

> **Reference:** [Sieve of Eratosthenes - GeeksforGeeks](https://www.geeksforgeeks.org/sieve-of-eratosthenes/)

---


In [9]:
def generate_primes_sieve(limit_count):
    """Generate the first `limit_count` prime numbers using the Sieve of Eratosthenes."""
    limit_estimate = 550  # Rough estimate to find first 100 primes
    sieve = [True] * (limit_estimate + 1)
    sieve[0:2] = [False, False]

    for i in range(2, int(math.isqrt(limit_estimate)) + 1):
        if sieve[i]:
            for j in range(i * i, limit_estimate + 1, i):
                sieve[j] = False

    primes = [i for i, is_prime in enumerate(sieve) if is_prime]
    return primes[:limit_count]

# First 100 primes using Sieve of Eratosthenes
primes_sieve = generate_primes_sieve(100)
print(primes_sieve)


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


### Comparison of Methods

| Method | Speed | Complexity | Use Case |
|--------|-------|------------|----------|
| Trial Division | Slow for large numbers | O(√n) per number | Simple checks for small values |
| Sieve of Eratosthenes | Very fast | O(n log log n) | Best for generating many primes |

Both methods successfully find the first 100 prime numbers, but the Sieve is significantly more efficient when searching larger ranges.

---


## Task 5: Roots

In this task, we calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

The steps are:
1. Take the square root of each prime number.
2. Extract the fractional part (i.e., remove the integer part).
3. Multiply the fractional part by 2^32 to get a 32-bit integer.
4. Collect and display the results.

This technique is relevant in cryptographic constants, such as the initial hash values in SHA256.

---


In [10]:
def fractional_part_32bit(number):
    """Calculate the first 32 bits of the fractional part of a square root."""
    root = math.sqrt(number)
    fractional_part = root - math.floor(root)
    return int(fractional_part * (2**32))

# Use the primes from Task 4
prime_roots_32bit = {}

for prime in primes_trial:  # You could also use primes_sieve if preferred
    prime_roots_32bit[prime] = fractional_part_32bit(prime)

# Display nicely
for prime, fraction_bits in prime_roots_32bit.items():
    print(f"Prime: {prime}, 32-bit Fraction: {fraction_bits} (Hex: {fraction_bits:08x})")


Prime: 2, 32-bit Fraction: 1779033703 (Hex: 6a09e667)
Prime: 3, 32-bit Fraction: 3144134277 (Hex: bb67ae85)
Prime: 5, 32-bit Fraction: 1013904242 (Hex: 3c6ef372)
Prime: 7, 32-bit Fraction: 2773480762 (Hex: a54ff53a)
Prime: 11, 32-bit Fraction: 1359893119 (Hex: 510e527f)
Prime: 13, 32-bit Fraction: 2600822924 (Hex: 9b05688c)
Prime: 17, 32-bit Fraction: 528734635 (Hex: 1f83d9ab)
Prime: 19, 32-bit Fraction: 1541459225 (Hex: 5be0cd19)
Prime: 23, 32-bit Fraction: 3418070365 (Hex: cbbb9d5d)
Prime: 29, 32-bit Fraction: 1654270250 (Hex: 629a292a)
Prime: 31, 32-bit Fraction: 2438529370 (Hex: 9159015a)
Prime: 37, 32-bit Fraction: 355462360 (Hex: 152fecd8)
Prime: 41, 32-bit Fraction: 1731405415 (Hex: 67332667)
Prime: 43, 32-bit Fraction: 2394180231 (Hex: 8eb44a87)
Prime: 47, 32-bit Fraction: 3675008525 (Hex: db0c2e0d)
Prime: 53, 32-bit Fraction: 1203062813 (Hex: 47b5481d)
Prime: 59, 32-bit Fraction: 2925498710 (Hex: ae5f9156)
Prime: 61, 32-bit Fraction: 3479995859 (Hex: cf6c85d3)
Prime: 67, 32-bi

### Explanation

For each prime number:
- The square root is calculated.
- The integer part is removed, leaving only the decimal fraction.
- The fractional part is scaled up by `2^32` to convert it into a 32-bit unsigned integer.

This method of extracting fractional parts is similar to how constants are derived for cryptographic algorithms like SHA256, where precision and randomness are critical.

---


> **Reference:** [FIPS 180-4, Secure Hash Standard (SHA256 Constants)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

The use of fractional parts of irrational numbers (like square roots and cube roots) is outlined in FIPS 180-4, which describes how initial SHA256 constants are derived.


## Task 6

## Task 7